In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import random
import time

import adai_optim

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/home/jukie/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/home/jukie/Project/eeg-eye-state-classification-master/eeg_eye_state.csv', skiprows = [i for i in range(19)], header=None)

In [3]:
X = df.loc[:,0:13]
y = df.loc[:,14:]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15
#, random_state=1234
)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:
print(f"Training Data has {len(X_train)} examples")
print(f"Testing Data has {len(X_test)} examples")

Training Data has 12717 examples
Testing Data has 2245 examples


In [7]:
y_train = y_train.to_numpy()


In [8]:
y_test =y_test.to_numpy()

In [9]:
## Train Data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## Test Data    
class testData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test),
                    torch.FloatTensor(y_test))

In [10]:
class accData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

acc_data = accData(torch.FloatTensor(X_test))

In [11]:
# Create DataLoaders for torch
BATCH_SIZE = 100
LEARNING_RATE=0.001
EPOCHS=2000

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE,)
acc_loader = DataLoader(dataset=acc_data, batch_size=1)

In [12]:
class eca_layer(nn.Module):
    """Constructs a ECA module.

    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, k_size=3):
        super(eca_layer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)

        # Two different branches of ECA module
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)

        # Multi-scale information fusion
        y = self.sigmoid(y)

        return x * y.expand_as(x)

In [13]:
class MLPNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.dim = nn.Linear(1, 128)
        self.layer_1 = nn.Linear(64, 128) 
        self.layer_2 = nn.Linear(128, 128)
        self.eca1 = eca_layer(3)
        self.eca2 = eca_layer(3)
        self.flat = nn.Sequential(
            nn.Linear(1792,128),
            #nn.Dropout(p=0.5),
            nn.ReLU())
        self.layer_out = nn.Linear(128, 1)
        
        self.leakyRelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.25)
        self.batchnorm1 = nn.BatchNorm1d(14)
        self.batchnorm2 = nn.BatchNorm1d(14)

        self.attblock = nn.Sequential(
            eca_layer(3),
            nn.Linear(128, 256),
            nn.LeakyReLU(),
            nn.BatchNorm1d(14),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.BatchNorm1d(14),
            #nn.Dropout(p=0.25)
        )

    def forward(self, x):
        x = torch.unsqueeze(x,2)
        x = self.dim(x)
        '''
        x = self.eca1(x)
        x = self.leakyRelu(self.layer_1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.leakyRelu(self.layer_2(x))
        x = self.batchnorm2(x)
        '''
        x = self.attblock(x)
        x = self.attblock(x)
        #x = self.attblock(x)
        x = self.eca2(x)
        #x = self.dropout(x)
        x = torch.flatten(x,1)
        x = self.flat(x)
        x = self.layer_out(x)
        return x

model = MLPNetwork()

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 320,762 trainable parameters


In [15]:
#optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=5e-3)
optimizer = adai_optim.Adai(model.parameters(), lr=LEARNING_RATE, betas=(0.1, 0.99), eps=1e-03, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda:0')

In [16]:
model = model.to(device)
criterion = criterion.to(device)

In [17]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [18]:
#model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    tepoch_loss = 0
    tepoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    model.eval()
    for X_batch, y_batch in test_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        
        y_pred = model(X_batch)
        tloss = criterion(y_pred, y_batch)
        tacc = accuracy(y_pred, y_batch)
                
        tepoch_loss += tloss.item()
        tepoch_acc += tacc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')    
    print(f'Epoch {e+0:03}: | tLoss: {tepoch_loss/len(test_loader):.5f} | tAcc: {tepoch_acc/len(test_loader):.3f}')

Epoch 001: | Loss: 0.69031 | Acc: 53.680
Epoch 001: | tLoss: 0.69129 | tAcc: 52.652
Epoch 002: | Loss: 0.68832 | Acc: 55.008
Epoch 002: | tLoss: 0.68882 | tAcc: 53.261
Epoch 003: | Loss: 0.68731 | Acc: 54.859
Epoch 003: | tLoss: 0.68819 | tAcc: 53.870
Epoch 004: | Loss: 0.68560 | Acc: 55.172
Epoch 004: | tLoss: 0.68559 | tAcc: 55.304
Epoch 005: | Loss: 0.68492 | Acc: 55.383
Epoch 005: | tLoss: 0.68399 | tAcc: 56.043
Epoch 006: | Loss: 0.68454 | Acc: 55.297
Epoch 006: | tLoss: 0.68269 | tAcc: 57.348
Epoch 007: | Loss: 0.68395 | Acc: 55.234
Epoch 007: | tLoss: 0.68139 | tAcc: 58.391
Epoch 008: | Loss: 0.68271 | Acc: 55.812
Epoch 008: | tLoss: 0.67970 | tAcc: 58.348
Epoch 009: | Loss: 0.68088 | Acc: 56.094
Epoch 009: | tLoss: 0.67900 | tAcc: 58.304
Epoch 010: | Loss: 0.68164 | Acc: 55.648
Epoch 010: | tLoss: 0.67856 | tAcc: 59.304
Epoch 011: | Loss: 0.68055 | Acc: 56.555
Epoch 011: | tLoss: 0.67672 | tAcc: 58.435
Epoch 012: | Loss: 0.67839 | Acc: 57.008
Epoch 012: | tLoss: 0.68238 | tAcc:

KeyboardInterrupt: 

In [19]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in acc_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [20]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.87      0.58      0.70      1233
           1       0.64      0.90      0.74      1012

    accuracy                           0.72      2245
   macro avg       0.75      0.74      0.72      2245
weighted avg       0.77      0.72      0.72      2245

